### vgg_model

In [1]:
""" Load VGGNet weights needed for the implementation of the paper 
"A Neural Algorithm of Artistic Style" by Gatys et al. in TensorFlow.

Author: Chip Huyen (huyenn@stanford.edu)
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
For more details, please read the assignment handout:
http://web.stanford.edu/class/cs20si/assignments/a2.pdf
"""

import numpy as np
import tensorflow as tf
import scipy.io

def _weights(vgg_layers, layer, expected_layer_name):
    """ Return the weights and biases already trained by VGG
    """
    W = vgg_layers[0][layer][0][0][2][0][0]
    b = vgg_layers[0][layer][0][0][2][0][1]
    layer_name = vgg_layers[0][layer][0][0][0][0]
    assert layer_name == expected_layer_name
    return W, b.reshape(b.size)

def _conv2d_relu(vgg_layers, prev_layer, layer, layer_name):
    """ Return the Conv2D layer with RELU using the weights, biases from the VGG
    model at 'layer'.
    Inputs:
        vgg_layers: holding all the layers of VGGNet
        prev_layer: the output tensor from the previous layer
        layer: the index to current layer in vgg_layers
        layer_name: the string that is the name of the current layer.
                    It's used to specify variable_scope.

    Output:
        relu applied on the convolution.

    Note that you first need to obtain W and b from vgg-layers using the function
    _weights() defined above.
    W and b returned from _weights() are numpy arrays, so you have
    to convert them to TF tensors using tf.constant.
    Note that you'll have to do apply relu on the convolution.
    Hint for choosing strides size: 
        for small images, you probably don't want to skip any pixel
    """
    with tf.variable_scope(layer_name) as scope:
        W, b = _weights(vgg_layers, layer, layer_name)
        W = tf.constant(W, name='weights')
        b = tf.constant(b, name='bias')
        conv2d = tf.nn.conv2d(prev_layer, filter=W, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2d + b)

def _avgpool(prev_layer):
    """ Return the average pooling layer. The paper suggests that average pooling
    actually works better than max pooling.
    Input:
        prev_layer: the output tensor from the previous layer

    Output:
        the output of the tf.nn.avg_pool() function.
    Hint for choosing strides and kszie: choose what you feel appropriate
    """
    return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                          padding='SAME', name='avg_pool_')

def load_vgg(path, input_image):
    """ Load VGG into a TensorFlow model.
    Use a dictionary to hold the model instead of using a Python class
    """
    vgg = scipy.io.loadmat(path)
    vgg_layers = vgg['layers']

    graph = {} 
    graph['conv1_1']  = _conv2d_relu(vgg_layers, input_image, 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(vgg_layers, graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _avgpool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(vgg_layers, graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(vgg_layers, graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _avgpool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(vgg_layers, graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(vgg_layers, graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(vgg_layers, graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(vgg_layers, graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(vgg_layers, graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(vgg_layers, graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(vgg_layers, graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(vgg_layers, graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(vgg_layers, graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(vgg_layers, graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(vgg_layers, graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(vgg_layers, graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    
    return graph

### googlenet_model

In [2]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

def load_googlenet(path, input_image):
    # Load GoogLeNet into a TensorFlow model.
    with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits, end_points = inception.inception_v3(input_image, num_classes=1001, is_training=False)
    return end_points

### utils

In [3]:
""" Utils needed for the implementation of the paper "A Neural Algorithm of Artistic Style"
by Gatys et al. in TensorFlow.

Author: Chip Huyen (huyenn@stanford.edu)
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
For more details, please read the assignment handout:
http://web.stanford.edu/class/cs20si/assignments/a2.pdf
"""
from __future__ import print_function

import os

from PIL import Image, ImageOps
import numpy as np
import scipy.misc
import tarfile
from six.moves import urllib

# VGG-19 parameters file
VGG_DOWNLOAD_LINK = 'http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat'
VGG_MODEL = 'imagenet-vgg-verydeep-19.mat'
EXPECTED_BYTES = 534904783

# GoogLeNet parameters file
GOOGLENET_DOWNLOAD_LINK = 'http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz'
GOOGLENET_MODEL = 'inception_v3.ckpt'
GOOGLENET_EXPECTED_BYTES = 100885009

def download(download_link, file_name, expected_bytes, model_type):
    """ Download the pretrained model if it's not already downloaded """
    if os.path.exists(file_name):
        print(model_type + " pre-trained model ready")
        return
    print("Downloading the " + model_type + " pre-trained model. This might take a while ...")
    suffix = download_link.rsplit('/', 1)[1]
    urllib.request.urlretrieve(download_link, suffix)
    file_stat = os.stat(suffix)
    if file_stat.st_size == expected_bytes:
        print('Successfully downloaded ' + model_type + ' pre-trained model', suffix)
    else:
        raise Exception('File ' + suffix +
                        ' might be corrupted. You should try downloading it with a browser.')
    if model_type == 'googlenet':
        tar = tarfile.open(suffix)
        tar.extractall()
        tar.close()
        os.remove(suffix)

def get_resized_image(img_path, height, width, save=True):
    image = Image.open(img_path)
    # it's because PIL is column major so you have to change place of width and height
    # this is stupid, i know
    image = ImageOps.fit(image, (width, height), Image.ANTIALIAS)
    if save:
        image_dirs = img_path.split('/')
        image_dirs[-1] = 'resized_' + image_dirs[-1]
        out_path = '/'.join(image_dirs)
        if not os.path.exists(out_path):
            image.save(out_path)
    image = np.asarray(image, np.float32)
    return np.expand_dims(image, 0)

def generate_noise_image(content_image, height, width, noise_ratio=0.6):
    noise_image = np.random.uniform(-20, 20, 
                                    (1, height, width, 3)).astype(np.float32)
    return noise_image * noise_ratio + content_image * (1 - noise_ratio)

def save_image(path, image):
    # Output should add back the mean pixels we subtracted at the beginning
    image = image[0] # the image
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)
    
def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

def load_model(model_type, input_image):
    global STYLE_LAYERS, CONTENT_LAYER
    if model_type == 'vgg':
        print('Loading VGG-19')
        STYLE_LAYERS = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']
        CONTENT_LAYER = 'conv4_2'
        download(VGG_DOWNLOAD_LINK, VGG_MODEL, EXPECTED_BYTES, 'vgg')
        return load_vgg(VGG_MODEL, input_image)
    if model_type == 'googlenet':    
        print('Loading GoogLeNet')
        #STYLE_LAYERS = ['Conv2d_1a_3x3', 'Conv2d_2a_3x3', 'Conv2d_3b_1x1', 'Conv2d_4a_3x3']
        #CONTENT_LAYER = 'Conv2d_2b_3x3'
        STYLE_LAYERS = ['MaxPool_3a_3x3', 'MaxPool_5a_3x3']
        CONTENT_LAYER = 'Mixed_6e'
        download(GOOGLENET_DOWNLOAD_LINK, GOOGLENET_MODEL, GOOGLENET_EXPECTED_BYTES, 'googlenet')
        return load_googlenet(GOOGLENET_MODEL, input_image)
    raise ValueError('No such model type')
    
def initialize_hyperparameters():
    if (MODEL_TYPE == GOOGLENET):
        W = [0.5, 1.0, 1.5, 3.0, 4.0]
        LR = 0.01
        CONTENT_WEIGHT = 0.0001
        STYLE_WEIGHT = 1
        ITERS = 300
    elif (MODEL_TYPE == VGG):
        W = [0.5, 1.0, 1.5, 3.0, 4.0]
        LR = 2
        CONTENT_WEIGHT = 0.01
        STYLE_WEIGHT = 1
        ITERS = 300
    else:
        raise ValueError('No such model type')
    return W, LR, CONTENT_WEIGHT, STYLE_WEIGHT, ITERS

### style_transfer

In [4]:
""" An implementation of the paper "A Neural Algorithm of Artistic Style"
by Gatys et al. in TensorFlow.

Author: Chip Huyen (huyenn@stanford.edu)
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
For more details, please read the assignment handout:
http://web.stanford.edu/class/cs20si/assignments/a2.pdf
"""
from __future__ import print_function

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import numpy as np
import tensorflow as tf

import shutil

# parameters to manage experiments

STYLE = 'wave'
CONTENT = 'max'

STYLE_IMAGE = 'styles/' + STYLE + '.jpg'
CONTENT_IMAGE = 'content/' + CONTENT + '.jpg'
IMAGE_HEIGHT = 299
IMAGE_WIDTH = 299
NOISE_RATIO = 0.6 # percentage of weight of the noise for intermixing with the content image

MEAN_PIXELS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
""" MEAN_PIXELS is defined according to description on their github:
https://gist.github.com/ksimonyan/211839e770f7b538e2d8
'In the paper, the model is denoted as the configuration D trained with scale jittering. 
The input images should be zero-centered by mean pixel (rather than mean image) subtraction. 
Namely, the following BGR values should be subtracted: [103.939, 116.779, 123.68].'
"""

GOOGLENET = 'googlenet'
VGG = 'vgg'

# model to run (GOOGLENET | VGG)
MODEL_TYPE = GOOGLENET

W, LR, CONTENT_WEIGHT, STYLE_WEIGHT, ITERS = initialize_hyperparameters()

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

def _create_content_loss(p, f):
    """ Calculate the loss between the feature representation of the
    content image and the generated image.
    
    Inputs: 
        p, f are just P, F in the paper 
        (read the assignment handout if you're confused)
        Note: we won't use the coefficient 0.5 as defined in the paper
        but the coefficient as defined in the assignment handout.
    Output:
        the content loss

    """
    return tf.reduce_sum((f - p) ** 2) / (4.0 * p.size)

def _gram_matrix(F, N, M):
    """ Create and return the gram matrix for tensor F
        Hint: you'll first have to reshape F
    """
    F = tf.reshape(F, (M, N))
    return tf.matmul(tf.transpose(F), F)

def _single_style_loss(a, g):
    """ Calculate the style loss at a certain layer
    Inputs:
        a is the feature representation of the real image
        g is the feature representation of the generated image
    Output:
        the style loss at a certain layer (which is E_l in the paper)

    Hint: 1. you'll have to use the function _gram_matrix()
        2. we'll use the same coefficient for style loss as in the paper
        3. a and g are feature representation, not gram matrices
    """
    N = a.shape[3] # number of filters
    M = a.shape[1] * a.shape[2] # height times width of the feature map
    A = _gram_matrix(a, N, M)
    G = _gram_matrix(g, N, M)
    return tf.reduce_sum((G - A) ** 2 / ((2 * N * M) ** 2))

def _create_style_loss(A, model):
    """ Return the total style loss
    """
    n_layers = len(STYLE_LAYERS)
    E = [_single_style_loss(A[i], model[STYLE_LAYERS[i]]) for i in range(n_layers)]
    
    return sum([W[i] * E[i] for i in range(n_layers)])

def _create_losses(model, input_image, content_image, style_image):
    with tf.variable_scope('loss') as scope:
        with tf.Session() as sess:
            if (MODEL_TYPE == GOOGLENET):
                saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='InceptionV3'))
                saver.restore(sess, GOOGLENET_MODEL)
            sess.run(input_image.assign(content_image)) # assign content image to the input variable
            p = sess.run(model[CONTENT_LAYER])
        content_loss = _create_content_loss(p, model[CONTENT_LAYER])

        with tf.Session() as sess:
            if (MODEL_TYPE == GOOGLENET):
                saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='InceptionV3'))
                saver.restore(sess, GOOGLENET_MODEL)
            sess.run(input_image.assign(style_image))
            A = sess.run([model[layer_name] for layer_name in STYLE_LAYERS])                              
        style_loss = _create_style_loss(A, model)
        total_loss = CONTENT_WEIGHT * content_loss + STYLE_WEIGHT * style_loss

    return content_loss, style_loss, total_loss

def _create_summary(model):
    """ Create summary ops necessary
        Hint: don't forget to merge them
    """
    with tf.name_scope('summaries'):
        tf.summary.scalar('content_loss', model['content_loss'])
        tf.summary.scalar('style_loss', model['style_loss'])
        tf.summary.scalar('total_loss', model['total_loss'])
        tf.summary.histogram('histogram_content_loss', model['content_loss'])
        tf.summary.histogram('histogram_style_loss', model['style_loss'])
        tf.summary.histogram('histogram_total_loss', model['total_loss'])
        return tf.summary.merge_all()

def train(model, generated_image, initial_image):
    """ Train your model.
    Don't forget to create folders for checkpoints and outputs.
    """
    skip_step = 1
    with tf.Session() as sess:
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter('graphs', sess.graph)
        sess.run(generated_image.assign(initial_image))
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path and MODEL_TYPE == VGG:
            saver.restore(sess, ckpt.model_checkpoint_path)
        initial_step = model['global_step'].eval()
        
        start_time = time.time()
        for index in range(initial_step, ITERS):
            if index >= 5 and index < 20:
                skip_step = 10
            elif index >= 20:
                skip_step = 20
            
            sess.run(model['optimizer'])
            if (index + 1) % skip_step == 0:
                gen_image, total_loss, summary = sess.run([generated_image, model['total_loss'], 
                                                             model['summary_op']])
                gen_image = gen_image + MEAN_PIXELS
                writer.add_summary(summary, global_step=index)
                print('Step {}\n   Sum: {:5.1f}'.format(index + 1, np.sum(gen_image)))
                print('   Loss: {:5.1f}'.format(total_loss))
                print('   Time: {}'.format(time.time() - start_time))
                start_time = time.time()

                filename = 'outputs/%d.png' % (index)
                save_image(filename, gen_image)

                if (index + 1) % 20 == 0:
                    saver.save(sess, 'checkpoints/style_transfer', index)

reset_graph()
shutil.rmtree('checkpoints')
    
with tf.variable_scope('input') as scope:
    # use variable instead of placeholder because we're training the intial image to make it
    # look like both the content image and the style image
    input_image = tf.Variable(np.zeros([1, IMAGE_HEIGHT, IMAGE_WIDTH, 3]), dtype=tf.float32)
    
make_dir('checkpoints')
make_dir('outputs')
model = load_model(MODEL_TYPE, input_image)
model['global_step'] = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

content_image = get_resized_image(CONTENT_IMAGE, IMAGE_HEIGHT, IMAGE_WIDTH)
content_image = content_image - MEAN_PIXELS
style_image = get_resized_image(STYLE_IMAGE, IMAGE_HEIGHT, IMAGE_WIDTH)
style_image = style_image - MEAN_PIXELS

# if (MODEL_TYPE == GOOGLENET):
#     # normalize between -1 and 1
#     content_image[0] /= 255
#     style_image[0] /= 255

model['content_loss'], model['style_loss'], model['total_loss'] = _create_losses(model, 
                                                input_image, content_image, style_image)
model['optimizer'] = tf.train.AdamOptimizer(LR).minimize(model['total_loss'], 
                                                        global_step=model['global_step'])
model['summary_op'] = _create_summary(model)

initial_image = generate_noise_image(content_image, IMAGE_HEIGHT, IMAGE_WIDTH, NOISE_RATIO)
train(model, input_image, initial_image)

Loading GoogLeNet
googlenet pre-trained model ready
INFO:tensorflow:Restoring parameters from inception_v3.ckpt
INFO:tensorflow:Restoring parameters from inception_v3.ckpt
Step 1
   Sum: 24438829.4
   Loss: 6492104524038144.0
   Time: 0.917562007904
Step 2
   Sum: 24439383.0
   Loss: 47345704960.0
   Time: 0.118243932724
Step 3
   Sum: 24439810.9
   Loss: 47345819648.0
   Time: 0.117037057877
Step 4
   Sum: 24440161.3
   Loss: 47345811456.0
   Time: 0.116274118423
Step 5
   Sum: 24440457.5
   Loss: 47345725440.0
   Time: 0.116801023483
Step 10
   Sum: 24441462.8
   Loss: 47292153856.0
   Time: 0.401615858078
Step 20
   Sum: 24442384.2
   Loss: 45448273920.0
   Time: 0.760804891586
Step 40
   Sum: 24442876.7
   Loss: 45174276096.0
   Time: 2.97391319275
Step 60
   Sum: 24442951.8
   Loss: 45147377664.0
   Time: 2.80926394463
Step 80
   Sum: 24442962.8
   Loss: 45144485888.0
   Time: 3.03690505028
Step 100
   Sum: 24442963.5
   Loss: 45144096768.0
   Time: 2.81753182411
Step 120
   Sum: 

Step 2100
   Sum: 24442975.9
   Loss: 2847754752.0
   Time: 3.07714986801
Step 2120
   Sum: 24442977.2
   Loss: 2836001792.0
   Time: 2.82394695282
Step 2140
   Sum: 24442978.5
   Loss: 2823334144.0
   Time: 3.07584190369
Step 2160
   Sum: 24442979.8
   Loss: 2775614720.0
   Time: 2.83420610428
Step 2180
   Sum: 24442981.0
   Loss: 2761930240.0
   Time: 3.10021114349
Step 2200
   Sum: 24442982.3
   Loss: 2751293440.0
   Time: 2.82227993011
Step 2220
   Sum: 24442983.5
   Loss: 2740907264.0
   Time: 2.84985399246
Step 2240
   Sum: 24442984.7
   Loss: 2730774016.0
   Time: 3.06107902527
Step 2260
   Sum: 24442985.8
   Loss: 2720785408.0
   Time: 2.8273730278
Step 2280
   Sum: 24442986.9
   Loss: 2711073024.0
   Time: 3.05846691132
Step 2300
   Sum: 24442987.9
   Loss: 2701430528.0
   Time: 2.84723496437
Step 2320
   Sum: 24442989.0
   Loss: 2691771904.0
   Time: 2.83301591873
Step 2340
   Sum: 24442990.0
   Loss: 2682158592.0
   Time: 3.14985990524
Step 2360
   Sum: 24442991.1
   Loss: 2

Step 4320
   Sum: 24442982.1
   Loss: 2474615040.0
   Time: 2.82065105438
Step 4340
   Sum: 24442982.1
   Loss: 2472819200.0
   Time: 2.84770679474
Step 4360
   Sum: 24442982.1
   Loss: 2473162240.0
   Time: 3.05579090118
Step 4380
   Sum: 24442982.1
   Loss: 2471569664.0
   Time: 2.81349396706
Step 4400
   Sum: 24442982.1
   Loss: 2471205376.0
   Time: 3.07915997505
Step 4420
   Sum: 24442982.1
   Loss: 2471072000.0
   Time: 2.83132505417
Step 4440
   Sum: 24442982.0
   Loss: 2471065600.0
   Time: 3.08940792084
Step 4460
   Sum: 24442982.0
   Loss: 2471067648.0
   Time: 2.85566306114
Step 4480
   Sum: 24442982.0
   Loss: 2472576000.0
   Time: 2.82024097443
Step 4500
   Sum: 24442982.0
   Loss: 2471310080.0
   Time: 3.06744194031
Step 4520
   Sum: 24442982.0
   Loss: 2471043328.0
   Time: 2.84003090858
Step 4540
   Sum: 24442982.0
   Loss: 2471009792.0
   Time: 3.07809305191
Step 4560
   Sum: 24442982.0
   Loss: 2470997760.0
   Time: 2.81447410583
Step 4580
   Sum: 24442982.0
   Loss: 

Step 6540
   Sum: 24442981.5
   Loss: 2471427072.0
   Time: 2.82418107986
Step 6560
   Sum: 24442981.5
   Loss: 2470687232.0
   Time: 3.09806108475
Step 6580
   Sum: 24442981.5
   Loss: 2474363392.0
   Time: 2.84624695778
Step 6600
   Sum: 24442981.5
   Loss: 2471097856.0
   Time: 2.81633400917
Step 6620
   Sum: 24442981.5
   Loss: 2470628864.0
   Time: 3.05529308319
Step 6640
   Sum: 24442981.5
   Loss: 2471696128.0
   Time: 2.83098506927
Step 6660
   Sum: 24442981.5
   Loss: 2471207424.0
   Time: 3.07671189308
Step 6680
   Sum: 24442981.5
   Loss: 2470490112.0
   Time: 2.8209271431
Step 6700
   Sum: 24442981.4
   Loss: 2470438912.0
   Time: 3.08136606216
Step 6720
   Sum: 24442981.4
   Loss: 2472504320.0
   Time: 2.80882406235
Step 6740
   Sum: 24442981.4
   Loss: 2471154176.0
   Time: 2.82437896729
Step 6760
   Sum: 24442981.4
   Loss: 2470599680.0
   Time: 3.06303310394
Step 6780
   Sum: 24442981.4
   Loss: 2470446336.0
   Time: 2.87287402153
Step 6800
   Sum: 24442981.4
   Loss: 2

Step 8760
   Sum: 24442971.4
   Loss: 2417072384.0
   Time: 2.90007305145
Step 8780
   Sum: 24442971.4
   Loss: 2416415488.0
   Time: 3.06775593758
Step 8800
   Sum: 24442971.3
   Loss: 2416414208.0
   Time: 2.82898402214
Step 8820
   Sum: 24442971.2
   Loss: 2416406784.0
   Time: 3.05983400345
Step 8840
   Sum: 24442971.1
   Loss: 2418448640.0
   Time: 2.83597993851
Step 8860
   Sum: 24442970.9
   Loss: 2416445440.0
   Time: 2.81726002693
Step 8880
   Sum: 24442970.7
   Loss: 2417514240.0
   Time: 3.06140613556
Step 8900
   Sum: 24442970.9
   Loss: 2417302016.0
   Time: 2.82377099991
Step 8920
   Sum: 24442970.8
   Loss: 2416431360.0
   Time: 3.05564212799
Step 8940
   Sum: 24442970.7
   Loss: 2416386560.0
   Time: 2.81791901588
Step 8960
   Sum: 24442970.6
   Loss: 2416503040.0
   Time: 3.08582997322
Step 8980
   Sum: 24442970.5
   Loss: 2422397440.0
   Time: 2.820125103
Step 9000
   Sum: 24442970.6
   Loss: 2418002688.0
   Time: 2.83960103989
Step 9020
   Sum: 24442970.5
   Loss: 24

...

Step 19820
   Sum: 24441883.1
   Loss: 853422720.0
   Time: 3.08657288551
Step 19840
   Sum: 24441881.7
   Loss: 821558912.0
   Time: 2.81862306595
Step 19860
   Sum: 24441918.5
   Loss: 810447680.0
   Time: 3.06484103203
Step 19880
   Sum: 24441913.3
   Loss: 809555328.0
   Time: 2.84182882309
Step 19900
   Sum: 24441903.2
   Loss: 809397120.0
   Time: 2.8369538784
Step 19920
   Sum: 24441894.8
   Loss: 809295744.0
   Time: 3.10934901237
Step 19940
   Sum: 24441889.5
   Loss: 809208896.0
   Time: 2.83821797371
Step 19960
   Sum: 24441887.3
   Loss: 809142912.0
   Time: 3.07065987587
Step 19980
   Sum: 24441885.5
   Loss: 809099648.0
   Time: 2.82336187363
Step 20000
   Sum: 24441883.7
   Loss: 809126400.0
   Time: 3.1144618988
Step 20020
   Sum: 24441950.4
   Loss: 812630336.0
   Time: 2.83772802353
Step 20040
   Sum: 24442143.8
   Loss: 813649408.0
   Time: 2.82221698761
Step 20060
   Sum: 24442149.8
   Loss: 809505152.0
   Time: 3.07747006416
Step 20080
   Sum: 24442127.5
   Loss: 8

Step 22040
   Sum: 24443890.6
   Loss: 808618880.0
   Time: 3.09981703758
Step 22060
   Sum: 24443875.5
   Loss: 808587200.0
   Time: 2.84775280952
Step 22080
   Sum: 24443865.0
   Loss: 808616704.0
   Time: 3.07929587364
Step 22100
   Sum: 24443857.1
   Loss: 808541184.0
   Time: 2.86080789566
Step 22120
   Sum: 24443852.0
   Loss: 808534784.0
   Time: 3.08579802513
Step 22140
   Sum: 24443849.8
   Loss: 808620032.0
   Time: 2.84515213966
Step 22160
   Sum: 24443940.3
   Loss: 850892480.0
   Time: 2.8328461647
Step 22180
   Sum: 24444117.8
   Loss: 811546048.0
   Time: 3.05505681038
Step 22200
   Sum: 24444141.0
   Loss: 808647744.0
   Time: 2.83563399315
Step 22220
   Sum: 24444117.5
   Loss: 808570624.0
   Time: 3.14092206955
Step 22240
   Sum: 24444100.9
   Loss: 808526720.0
   Time: 2.82713389397
Step 22260
   Sum: 24444093.3
   Loss: 808513920.0
   Time: 3.09775495529
Step 22280
   Sum: 24444089.0
   Loss: 808510144.0
   Time: 2.824614048
Step 22300
   Sum: 24444085.9
   Loss: 80

Step 24260
   Sum: 24447492.1
   Loss: 815829312.0
   Time: 2.83343410492
Step 24280
   Sum: 24447511.1
   Loss: 809341824.0
   Time: 3.0760409832
Step 24300
   Sum: 24447512.8
   Loss: 808476928.0
   Time: 2.81067299843
Step 24320
   Sum: 24447502.6
   Loss: 808402624.0
   Time: 2.85806703568
Step 24340
   Sum: 24447495.4
   Loss: 808390784.0
   Time: 3.15552282333
Step 24360
   Sum: 24447491.1
   Loss: 808388160.0
   Time: 2.82376909256
Step 24380
   Sum: 24447487.5
   Loss: 808429056.0
   Time: 3.0687379837
Step 24400
   Sum: 24447483.0
   Loss: 808745152.0
   Time: 2.83655786514
Step 24420
   Sum: 24447516.9
   Loss: 848904512.0
   Time: 3.07746911049
Step 24440
   Sum: 24447780.9
   Loss: 809777280.0
   Time: 2.8227891922
Step 24460
   Sum: 24447811.1
   Loss: 808894336.0
   Time: 3.09222912788
Step 24480
   Sum: 24447796.8
   Loss: 808471104.0
   Time: 2.85821890831
Step 24500
   Sum: 24447784.9
   Loss: 808391360.0
   Time: 2.84807705879
Step 24520
   Sum: 24447778.1
   Loss: 80

Step 26480
   Sum: 24452537.7
   Loss: 852587904.0
   Time: 2.85033297539
Step 26500
   Sum: 24452931.5
   Loss: 815417792.0
   Time: 3.09405994415
Step 26520
   Sum: 24453101.2
   Loss: 810216896.0
   Time: 2.83057904243
Step 26540
   Sum: 24454008.4
   Loss: 940556608.0
   Time: 3.06373786926
Step 26560
   Sum: 24455336.4
   Loss: 826463616.0
   Time: 2.86085391045
Step 26580
   Sum: 24455617.2
   Loss: 815684416.0
   Time: 3.09725880623
Step 26600
   Sum: 24455703.6
   Loss: 812116672.0
   Time: 2.83140587807
Step 26620
   Sum: 24455687.6
   Loss: 811264960.0
   Time: 3.0762629509
Step 26640
   Sum: 24455680.9
   Loss: 811107200.0
   Time: 2.8251311779
Step 26660
   Sum: 24455676.3
   Loss: 810972800.0
   Time: 2.83033895493
Step 26680
   Sum: 24455677.2
   Loss: 810035200.0
   Time: 3.09005498886
Step 26700
   Sum: 24455672.9
   Loss: 809902016.0
   Time: 2.86350178719
Step 26720
   Sum: 24455669.5
   Loss: 809779968.0
   Time: 3.09354305267
Step 26740
   Sum: 24455668.5
   Loss: 8

Step 28700
   Sum: 24459908.0
   Loss: 808856384.0
   Time: 3.09084892273
Step 28720
   Sum: 24459917.7
   Loss: 808851584.0
   Time: 2.86933588982
Step 28740
   Sum: 24459924.0
   Loss: 808890496.0
   Time: 3.06535005569
Step 28760
   Sum: 24459818.9
   Loss: 916305408.0
   Time: 2.81438994408
Step 28780
   Sum: 24460264.7
   Loss: 812867136.0
   Time: 2.81924796104
Step 28800
   Sum: 24460353.8
   Loss: 809649088.0
   Time: 3.09081697464
Step 28820
   Sum: 24460416.8
   Loss: 808904960.0
   Time: 2.8351328373
Step 28840
   Sum: 24460440.2
   Loss: 808886144.0
   Time: 3.05752396584
Step 28860
   Sum: 24460453.8
   Loss: 808859776.0
   Time: 2.81476402283
Step 28880
   Sum: 24460465.3
   Loss: 808854848.0
   Time: 3.08332920074
Step 28900
   Sum: 24460475.1
   Loss: 808850368.0
   Time: 2.82743000984
Step 28920
   Sum: 24460483.6
   Loss: 808846272.0
   Time: 2.86218690872
Step 28940
   Sum: 24460499.9
   Loss: 809199680.0
   Time: 3.05032610893
Step 28960
   Sum: 24460633.9
   Loss: 

In [5]:
model

{'AuxLogits': <tf.Tensor 'InceptionV3/AuxLogits/SpatialSqueeze:0' shape=(1, 1001) dtype=float32>,
 'Conv2d_1a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_1a_3x3/Relu:0' shape=(1, 149, 149, 32) dtype=float32>,
 'Conv2d_2a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2a_3x3/Relu:0' shape=(1, 147, 147, 32) dtype=float32>,
 'Conv2d_2b_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2b_3x3/Relu:0' shape=(1, 147, 147, 64) dtype=float32>,
 'Conv2d_3b_1x1': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_3b_1x1/Relu:0' shape=(1, 73, 73, 80) dtype=float32>,
 'Conv2d_4a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_4a_3x3/Relu:0' shape=(1, 71, 71, 192) dtype=float32>,
 'Logits': <tf.Tensor 'InceptionV3/Logits/SpatialSqueeze:0' shape=(1, 1001) dtype=float32>,
 'MaxPool_3a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_3a_3x3/MaxPool:0' shape=(1, 73, 73, 64) dtype=float32>,
 'MaxPool_5a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_5a_3x3/MaxPool:0' shape=(1, 35, 35, 192) dtype=floa